In [19]:
import GPy
import gpflow
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
X = np.arange(1, 3, 0.5)[None].T
Xprime = np.arange(3, 5, 0.5)[None].T

In [35]:
linear_gpy = GPy.kern.Linear(1)
linear_gpy.variances[:] = 3.123
linear_gpflow = gpflow.kernels.Linear(1)
linear_gpflow.variance.assign(3.123)

(linear_gpy.K(X, Xprime) - linear_gpflow.K(X, Xprime).numpy()).sum()

-1.7763568394002505e-15

In [40]:
rbf_gpy = GPy.kern.RBF(1)
rbf_gpy.variance[:] = 1.123
rbf_gpy.lengthscale[:] = 3.333

rbf_gpflow = gpflow.kernels.SquaredExponential(1)
rbf_gpflow.variance.assign(1.123)
rbf_gpflow.lengthscales.assign(3.333)

(rbf_gpy.K(X, Xprime) - rbf_gpflow.K(X, Xprime).numpy()).sum()

7.771561172376096e-16

In [52]:
per_gpy = GPy.kern.StdPeriodic(1)
per_gpy.variance[:] = 5.5555
per_gpy.lengthscale[:] = 1.111
per_gpy.period[:] = 3.333

per_gpflow = gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential())
per_gpflow.base_kernel.variance.assign(5.5555)
per_gpflow.base_kernel.lengthscales.assign(1.111)
per_gpflow.period.assign(3.333)

(per_gpy.K(X, Xprime) - per_gpflow.K(X, Xprime).numpy()).sum()

-7.993605777301127e-15

In [56]:
cos_gpy = GPy.kern.Cosine(1)
cos_gpy.variance[:] = 5.5555
cos_gpy.lengthscale[:] = 1.111

cos_gpflow = gpflow.kernels.Cosine(1)
cos_gpflow.variance.assign(5.5555)
cos_gpflow.lengthscales.assign(1.111)

(cos_gpy.K(X, Xprime) - cos_gpflow.K(X, Xprime).numpy()).sum()

-16.179879237675678

In [62]:
from gpflow.kernels import AnisotropicStationary
import tensorflow as tf

class CosineLikeGPy(AnisotropicStationary):
    """
    The Cosine kernel. Functions drawn from a GP with this kernel are sinusoids
    (with a random phase).  The kernel equation is

        k(r) = σ² cos{2πd}

    where:
    d  is the sum of the per-dimension differences between the input points, scaled by the
    lengthscale parameter ℓ (i.e. Σᵢ [(X - X2ᵀ) / ℓ]ᵢ),
    σ² is the variance parameter.
    """

    def K_d(self, d):
        d = tf.reduce_sum(d, axis=-1)
        return self.variance * tf.cos(d) #2 * np.pi *

In [63]:
cos_gpy = GPy.kern.Cosine(1)
cos_gpy.variance[:] = 5.5555
cos_gpy.lengthscale[:] = 1.111

cosgpy_gpflow = CosineLikeGPy(1)
cosgpy_gpflow.variance.assign(5.5555)
cosgpy_gpflow.lengthscales.assign(1.111)

(cos_gpy.K(X, Xprime) - cosgpy_gpflow.K(X, Xprime).numpy()).sum()

3.774758283725532e-15

In [64]:
import model as m
import model_gpflow as mflow

[SCIPY OPT: ] False


In [66]:
sm = m.ExtraKernels.SpectralMixture(2)
smflow = mflow.ExtraKernels.SpectralMixture(2)
(sm.K(X, Xprime) - smflow.K(X, Xprime).numpy()).sum()

1.441132767808495e-06